# Question 3 - Gradient Systems Example
This notebook is a toy to visualise an example of a gradient system

Import required libraries

In [ ]:
import numpy as np
from scipy.integrate import odeint  # Used to numerically solve the ODE
import matplotlib.pyplot as plt

In [ ]:
# Library to allow for interactive plots
from IPython.display import display
from ipywidgets import interact, widgets, interactive

A gradient system is of the form: $\dot{\vec{x}}=-\nabla V(\vec{x});~ x\in\mathbb{R}^n,~ V:\mathbb{R}^n\rightarrow \mathbb{R}$

In this notebook we are working with a 2-dim system, so $x\in\mathbb{R}^2,~ V:\mathbb{R}^2\rightarrow \mathbb{R}$

Define the function $-\nabla V(\vec{x})$

In [ ]:
def V(x):
    return np.sin(x[0]) * np.sin(x[1])

def gradient_V(x):
    return np.array([np.cos(x[0]) * np.sin(x[1]), np.sin(x[0]) * np.cos(x[1])])

### Visualising the gradient

In [ ]:
x1, x2 = np.linspace(-np.pi, np.pi, 100), np.linspace(-np.pi, np.pi, 100)
X1, X2 = np.meshgrid(x1, x2)
z = V([X1, X2])
dz_dx1, dz_dx2 = gradient_V([X1, X2])

In [ ]:
fig = plt.figure(figsize=(20, 10))

# 1st subplot: 3D surface plot
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(X1, X2, z, cmap='viridis', edgecolor='none', alpha=0.5)
ax1.contour(X1, X2, z, 10, colors='black', linestyles='solid')
ax1.set_xlabel('X1')
ax1.set_ylabel('X2')
ax1.set_zlabel('V')

ax1.view_init(elev=50, azim=-90)

# 2nd subplot: 2D vector field with contour lines
ax2 = fig.add_subplot(122)
res = 5
ax2.quiver(X1[::res, ::res], X2[::res, ::res], dz_dx1[::res, ::res], dz_dx2[::res, ::res], z[::res, ::res], cmap='viridis')
contour = ax2.contour(X1[::res, ::res], X2[::res, ::res], z[::res, ::res], 10, colors='black', linestyles='dashed')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.set_xlabel('X1 axis')
ax2.set_ylabel('X2 axis')

plt.show()

In this case we can analytically find the points where $\nabla V(\vec{x})=0$: 

$$(x1,~x2)=(\frac{n_1}{2}\pi, \frac{n_2}{2}\pi);~ n_1,~n_2\in\mathbb{Z}\ \ \ \&\ \ \  x_1=\pm x_2$$

For this example we limit $-1\le n_1,n_2\le 1$

In [ ]:
def V_zeros():
    zeros = list()
    for i in range(-1, 2):
        for j in range(-1, 2):
            if abs(i) == abs(j):
                zeros.append([i * np.pi/2, j * np.pi/2])
    return zeros

We plot these fixed points on the vector field

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# 2nd subplot: 2D vector field with contour lines
res = 5
ax.quiver(X1[::res, ::res], X2[::res, ::res], dz_dx1[::res, ::res], dz_dx2[::res, ::res], z[::res, ::res], cmap='viridis')
contour = ax.contour(X1[::res, ::res], X2[::res, ::res], z[::res, ::res], 10, colors='black', linestyles='dashed')
# plot FP
for fp in V_zeros():
    fp_x1, fp_x2 = fp
    ax.scatter(fp_x1, fp_x2, c='r', marker='x', s=200)

ax.clabel(contour, inline=True, fontsize=8)
ax.set_xlabel('X1 axis')
ax.set_ylabel('X2 axis')

plt.show()

# Trajectories

We define a function that `odeint` will recognise

In [ ]:
def gradient_system(x, t):
    return - gradient_V(x)

We start with a few random initial contidions and see how these evolve.

In [ ]:
# random points for initial conditions
# trajectories
num_traj = 15
initial_conditions = 2 * np.pi * (np.random.rand(num_traj, 2) - 1/2)

I specifically set an initial condition close to the saddle point:

In [ ]:
initial_conditions = np.vstack((initial_conditions, [0.11, 0.1]))

We now integrate the system for each initial condition

In [ ]:
traj = list()
for ic in initial_conditions:
    traj.append(odeint(gradient_system, ic, np.linspace(0, 1000, 10000)))

And now plot the trajectories. Note I have reversed the gradient here!

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))

# 2nd subplot: 2D vector field with contour lines
res = 5
ax.quiver(X1[::res, ::res], X2[::res, ::res], -dz_dx1[::res, ::res], -dz_dx2[::res, ::res], z[::res, ::res], cmap='viridis')
contour = ax.contour(X1[::res, ::res], X2[::res, ::res], z[::res, ::res], 10, colors='black', linestyles='dashed')
# plot FP
for fp in V_zeros():
    fp_x1, fp_x2 = fp
    ax.scatter(fp_x1, fp_x2, c='r', marker='x', s=200)
    

# plot the trajectories
for t in traj:
    ax.plot(t[:,0], t[:,1])
    
# plot the ICs
for ic in initial_conditions:
    ax.scatter(ic[0], ic[1])

ax.clabel(contour, inline=True, fontsize=8)
ax.set_xlabel('X1 axis')
ax.set_ylabel('X2 axis')

plt.show()